In [1]:
# Install required libraries
!pip install requests pandas plotly python-dotenv -q

print("✅ Libraries installed successfully!")

✅ Libraries installed successfully!


In [2]:
import requests
from google.colab import userdata
import pandas as pd
from datetime import datetime

# 1. Retrieve the keys we just saved
APP_ID = userdata.get('ADZUNA_APP_ID')
APP_KEY = userdata.get('ADZUNA_APP_KEY')

# 2. Define the API URL for US jobs
base_url = "https://api.adzuna.com/v1/api/jobs/us/search/1"

# 3. specific search parameters
params = {
    'app_id': APP_ID,
    'app_key': APP_KEY,
    'what': 'data analyst',   # The job title we want
    'results_per_page': 5,    # Just get 5 jobs to test
    'content-type': 'application/json'
}

# 4. Make the request
response = requests.get(base_url, params=params)

# 5. Check if it worked
if response.status_code == 200:
    data = response.json()
    total_jobs = data.get('count', 0)
    print(f"✅ SUCCESS! API Connected.")
    print(f"📊 Total Data Analyst jobs found in US: {total_jobs}")
    print("First job title found:", data['results'][0]['title'])
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

✅ SUCCESS! API Connected.
📊 Total Data Analyst jobs found in US: 16677
First job title found: Data Analyst – Procurement & Quality


In [3]:
import time

def fetch_all_jobs(query='data analyst', pages=10):
    """Fetch multiple pages of job data from Adzuna"""
    all_jobs = []

    print(f"🚀 Starting collection of {pages} pages of jobs...")

    for page in range(1, pages + 1):
        # API URL for specific page
        url = f"https://api.adzuna.com/v1/api/jobs/us/search/{page}"

        params = {
            'app_id': APP_ID,
            'app_key': APP_KEY,
            'what': query,
            'results_per_page': 50,  # Max allowed per call
            'content-type': 'application/json'
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                results = data.get('results', [])
                all_jobs.extend(results)
                print(f"   📄 Page {page}/{pages} fetched: {len(results)} jobs added")
            else:
                print(f"   ⚠️ Error on page {page}: Status {response.status_code}")

            # Sleep briefly to be nice to the API
            time.sleep(1)

        except Exception as e:
            print(f"   ❌ Critical error on page {page}: {e}")

    print(f"\n✅ COLLECTION COMPLETE: {len(all_jobs)} total jobs collected.")
    return all_jobs

# Run the function to get 500 jobs
raw_jobs = fetch_all_jobs(query='data analyst', pages=10)

🚀 Starting collection of 10 pages of jobs...
   📄 Page 1/10 fetched: 50 jobs added
   📄 Page 2/10 fetched: 50 jobs added
   📄 Page 3/10 fetched: 50 jobs added
   📄 Page 4/10 fetched: 50 jobs added
   📄 Page 5/10 fetched: 50 jobs added
   📄 Page 6/10 fetched: 50 jobs added
   📄 Page 7/10 fetched: 50 jobs added
   📄 Page 8/10 fetched: 50 jobs added
   📄 Page 9/10 fetched: 50 jobs added
   📄 Page 10/10 fetched: 50 jobs added

✅ COLLECTION COMPLETE: 500 total jobs collected.


In [5]:
def process_jobs_to_dataframe(jobs):
    """Convert raw API data to a structured DataFrame"""
    processed = []

    for job in jobs:
        # Extract location details safely
        loc = job.get('location', {})
        area = loc.get('area', [])

        # Try to find city and state (area list varies in length)
        city = area[-1] if len(area) > 0 else 'Unknown'
        state = area[1] if len(area) > 1 else 'Unknown'

        processed.append({
            'Title': job.get('title'),
            'Company': job.get('company', {}).get('display_name', 'Unknown'),
            'City': city,
            'State': state,
            'Salary_Min': job.get('salary_min'),
            'Salary_Max': job.get('salary_max'),
            'Description': job.get('description', ''),
            'Date_Posted': job.get('created'),
            'URL': job.get('redirect_url')
        })

    df = pd.DataFrame(processed)

    # Calculate an estimated 'Average Salary' for each job
    # (If min/max are missing, this becomes NaN, which is fine)
    df['Avg_Salary'] = (df['Salary_Min'] + df['Salary_Max']) / 2

    return df

# Process the data
jobs_df = process_jobs_to_dataframe(raw_jobs)

# Show the first few rows and the shape of the data
print(f"📊 DataFrame Shape: {jobs_df.shape[0]} rows x {jobs_df.shape[1]} columns")
jobs_df.head()

📊 DataFrame Shape: 500 rows x 10 columns


,Title,Company,City,State,Salary_Min,Salary_Max,Description,Date_Posted,URL,Avg_Salary
0,Data Analyst – Procurement & Quality,Carpenter Technology USA,Greenfield Manor,Pennsylvania,115383.13,115383.13,Carpenter Technology Corporation is a leading ...,2026-02-05T13:28:29Z,https://www.adzuna.com/land/ad/5615826956?se=h...,115383.13
1,Data Analyst,Guidehouse,Westlake,Maryland,131132.33,131132.33,Job Family : Data Science Consulting Travel Re...,2026-02-09T00:16:33Z,https://www.adzuna.com/land/ad/5621189366?se=h...,131132.33
2,Health Care Data Analyst,University Health,San Antonio,Texas,64388.79,64388.79,POSITION SUMMARY/RESPONSIBILITIES Responsible ...,2026-02-09T07:06:26Z,https://www.adzuna.com/land/ad/5621388503?se=h...,64388.79
3,Data Analyst,ProKatchers,Streetsboro,Ohio,92160.76,92160.76,Job Title : Data Analyst/ Data Coordinator/Dat...,2026-02-07T07:26:28Z,https://www.adzuna.com/land/ad/5619035534?se=h...,92160.76
4,Data Analyst,Jobot,Las Vegas,Nevada,86019.84,86019.84,Our client in Las Vegas is looking to hire a D...,2026-02-02T20:38:48Z,https://www.adzuna.com/land/ad/5611162422?se=h...,86019.84


In [6]:
import re

# 1. Define the skills we want to track
SKILLS_DATABASE = {
    'Programming': ['python', 'r', 'sql', 'java', 'scala', 'c++'],
    'Databases': ['mysql', 'postgresql', 'mongodb', 'oracle', 'sql server', 'snowflake', 'redshift'],
    'Cloud': ['aws', 'azure', 'google cloud', 'gcp'],
    'Visualization': ['tableau', 'power bi', 'looker', 'qlik', 'matplotlib', 'seaborn'],
    'Analysis': ['excel', 'statistics', 'machine learning', 'a/b testing', 'modeling'],
    'Big Data': ['spark', 'hadoop', 'kafka', 'databricks']
}

# Flatten the list for easier searching
ALL_SKILLS = [skill for category in SKILLS_DATABASE.values() for skill in category]

print(f"📚 Loaded {len(ALL_SKILLS)} skills to search for.")

📚 Loaded 32 skills to search for.


In [7]:
def extract_skills(text):
    """Scan text and return a list of found skills"""
    if not isinstance(text, str):
        return []

    found_skills = []
    text = text.lower() # Convert to lowercase so 'SQL' matches 'sql'

    for skill in ALL_SKILLS:
        # We use regex (\b) to ensure we match "SQL" but not "MySQL" accidentally
        # or "Java" but not "Javascript" (unless we want to)
        pattern = r'\b' + re.escape(skill) + r'\b'
        if re.search(pattern, text):
            found_skills.append(skill)

    return found_skills

# 2. Apply this function to every job in our table
print("🔍 Scanning job descriptions... this might take a moment...")
jobs_df['Skills_Found'] = jobs_df['Description'].apply(extract_skills)

# 3. Count up the results
all_skills_list = [skill for skills in jobs_df['Skills_Found'] for skill in skills]
from collections import Counter
skill_counts = Counter(all_skills_list)

# 4. Convert to a nice table
skills_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Count'])
skills_df = skills_df.sort_values('Count', ascending=False).reset_index(drop=True)

# Calculate percentage (frequency)
skills_df['Percentage'] = (skills_df['Count'] / len(jobs_df) * 100).round(1)

print("\n🏆 TOP 10 MOST IN-DEMAND SKILLS:\n")
print(skills_df.head(10).to_string(index=False))

🔍 Scanning job descriptions... this might take a moment...

🏆 TOP 10 MOST IN-DEMAND SKILLS:

    Skill  Count  Percentage
      sql     22         4.4
    azure     14         2.8
    excel      9         1.8
   oracle      8         1.6
  tableau      7         1.4
 power bi      7         1.4
 modeling      6         1.2
snowflake      2         0.4
   python      2         0.4
     java      2         0.4


In [8]:
# 1. Filter out jobs that didn't list a salary
salary_df = jobs_df[jobs_df['Avg_Salary'].notna()].copy()

print(f"💰 Analyzing salary data from {len(salary_df)} jobs...")

# 2. Group by City and calculate the average
city_stats = salary_df.groupby('City').agg({
    'Avg_Salary': 'mean',
    'Title': 'count' # Count how many jobs in that city
}).reset_index()

# 3. Rename columns for clarity
city_stats.rename(columns={'Title': 'Job_Count'}, inplace=True)

# 4. Sort by Highest Salary
top_cities = city_stats.sort_values('Avg_Salary', ascending=False).head(10)

# Format the salary to look like currency ($)
pd.options.display.float_format = '${:,.2f}'.format

print("\n💵 TOP 10 HIGHEST PAYING CITIES (in this dataset):\n")
print(top_cities[['City', 'Avg_Salary', 'Job_Count']].to_string(index=False))

💰 Analyzing salary data from 500 jobs...

💵 TOP 10 HIGHEST PAYING CITIES (in this dataset):

          City  Avg_Salary  Job_Count
Portola Valley $278,141.90          1
     Willernie $246,721.55          1
     Grandview $220,611.44          1
 Lake Campbell $204,159.72          1
  Circle Pines $199,274.84          1
     Shoreview $185,542.03          1
    Stillwater $184,443.28          1
       Mendota $179,205.53          1
        Prince $177,524.44          1
          Aiea $175,966.12          1


In [9]:
import plotly.express as px

# --- Chart 1: Top 15 Skills ---
fig_skills = px.bar(
    skills_df.head(15),
    x='Count',
    y='Skill',
    title='<b>Top 15 Most In-Demand Data Analyst Skills</b>',
    labels={'Count': 'Mentions in Job Ads'},
    color='Count',
    color_continuous_scale='Viridis',
    text='Count',
    template='plotly_dark'
)
fig_skills.update_layout(yaxis=dict(autorange="reversed"))
fig_skills.show()

# --- Chart 2: Salary Distribution ---
fig_salary = px.histogram(
    jobs_df,
    x='Avg_Salary',
    nbins=20,
    title='<b>Salary Distribution (The Bell Curve)</b>',
    color_discrete_sequence=['#00CC96'],
    template='plotly_dark'
)
# Add a line for the average
avg_sal = jobs_df['Avg_Salary'].mean()
fig_salary.add_vline(x=avg_sal, line_dash="dash", line_color="white", annotation_text=f"Avg: ${avg_sal:,.0f}")
fig_salary.show()

In [10]:
# Save the data to CSV files
from datetime import date

today = date.today().strftime("%Y-%m-%d")

# 1. Save the main job list
jobs_df.to_csv(f'raw_job_data_{today}.csv', index=False)

# 2. Save the skills analysis
skills_df.to_csv(f'skills_analysis_{today}.csv', index=False)

# 3. Save the city salary analysis
city_stats.to_csv(f'city_salary_stats_{today}.csv', index=False)

print("✅ Files saved to Colab!")

# Trigger the download to your computer
from google.colab import files

print("⬇️ Downloading files now...")
files.download(f'raw_job_data_{today}.csv')
files.download(f'skills_analysis_{today}.csv')
files.download(f'city_salary_stats_{today}.csv')

✅ Files saved to Colab!
⬇️ Downloading files now...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>